In [3]:
import sys
sys.path.append('../')
import seaborn as sns
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from dft_descriptors.prepocessing import preprocess
from classic_descriptors.featurisation import one_hot_encoding
from classic_descriptors.featurisation import process_yield
from rdkit import Chem
from sklearn.decomposition import PCA

import matplotlib

font = {'family' : 'normal',
        'size'   : 25}
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
matplotlib.rcParams['axes.unicode_minus'] = False
sns.set_style("white")
%matplotlib inline

In [4]:
# data-set downloaded from 10.1126/science.aar5169
df_hte = pd.read_csv("BH_HTE_scaled_3.csv")

In [5]:
# scope-like descriptors of maximum increase in mean square error 
# (ref : fig S23 10.1126/science.aar5169)
L_scope_reduced = [
 'aryl_halide_.C3_NMR_shift',
 'aryl_halide_.H2_electrostatic_charge',
 'aryl_halide_V2_frequency']

In [6]:
# optimization-like descriptors of maximum increase in mean square error 
# (ref : fig S23 10.1126/science.aar5169)
L_opt_reduced = ['additive_.C3_NMR_shift',
         'additive_E_LUMO',
         'additive_.O1_electrostatic_charge',
         'additive_.C5_electrostatic_charge',
         'additive_dipole_moment',
         'additive_molecular_volume',
         'base_electronegativity',
         'additive_E_HOMO',
         'additive_V1_intensity',
         'additive_.C4_NMR_shift',
         'additive_V1_frequency',
         'additive_surface_area',
         'additive_.C4_electrostatic_charge',
         'additive_.C3_electrostatic_charge',
         'additive_.N1_electrostatic_charge',
         'base_.N1_electrostatic_charge',
         'base_E_HOMO',
         'base_molecular_weight',
         'additive_ovality',
         'additive_hardness',
         'base_dipole_moment',
         'base_molecular_volume',
         'base_E_LUMO']

In [7]:
# Creating dataframe for HTE-Buchwald Hartwig  
# with projections on scope and optimisation
df_hte_sco = df_hte[L_scope_reduced]
df_hte_opt = df_hte[L_opt_reduced]
X_sco = df_hte_sco.values
X_opt = df_hte_opt.values

pca = PCA(n_components=2)
pca.fit(X_sco)
X_sco_pc1 = pca.transform(X_sco)
sco_pc1 = np.array([i[0] for i in X_sco_pc1])

pca2 = PCA(n_components=2)
pca2.fit(X_opt)
X_opt_pc1 = pca2.transform(X_opt)
opt_pc1 = np.array([i[1] for i in X_opt_pc1])

data_hte = pd.DataFrame(data=opt_pc1.reshape(-1,1), columns=["opt"])
data_hte["sco"] = sco_pc1.reshape(-1,1)
data_hte["yield"] = df_hte["yield"]

In [10]:
from dft_descriptors.featurisation import process_dataframe_dft

# load NiCOlit dataset
df = pd.read_csv("data_csv/Data_test11262021.csv")

# preprocessing and dft-featurization
def AL_preprocess(df):
    df["Lewis Acid"] = df["Lewis Acid"].fillna('NoLewisAcid')
    df["Lewis Acid"] = df["Lewis Acid"].replace('nan', 'NoLewisAcid')
    Lewis_Acids_to_drop = ['O=C(O[Cs])O[Cs]', 'Cl[Cs]', 
                       'O=S(=O)(O[Sc](OS(=O)(=O)C(F)(F)F)OS(=O)(=O)C(F)(F)F)C(F)(F)F', 
                       'F[Cs]', 'O=P(O[Na])(O[Na])O[Na]', '[Rb+]',
                       'CC(C)(C)C(=O)O[Cs]', '[Cs+]', 'CC(=O)O[Cu]OC(C)=O', 'F[Sr]F']
    for al in Lewis_Acids_to_drop:
        df = df[df["Lewis Acid"] != al]  
    return df.reset_index(drop=True)

# Creating dataframe for NiCOlit projection on scope and optimisation
df_lit = preprocess(df)
df_lit = AL_preprocess(df_lit)
X, y, DOIs, Mecas, Origin, (v_scope, v_optim) = process_dataframe_dft(df_lit, data_path='data_csv/', dim=True)
scope = np.array(v_scope)
optim = np.array(v_optim)

# the first 134 dimension corresponds to the scope dimension
X_sco = X[:, :np.sum(v_scope)]
# the other dimension corresponds to the optimization dimension
X_opt = X[:, np.sum(v_scope):]


# Projection on PCA 1 of scope and optimization
pca = PCA(n_components=2)
pca.fit(X_sco)
X_sco_pc1 = pca.transform(X_sco)
sco_pc1 = np.array([i[0] for i in X_sco_pc1])

pca2 = PCA(n_components=2)
pca2.fit(X_opt)
X_opt_pc1 = pca2.transform(X_opt)
opt_pc1 = np.array([i[1] for i in X_opt_pc1])

df_lit["proj scope"] = sco_pc1
df_lit["proj optimisation"] = opt_pc1
df_lit["yield"] = y
df_lit["data type"] = Origin

# Selecting one publication for the plot
doi_num = 13
dois = df_lit["DOI"].unique()
data_doi = df_lit[df_lit["DOI"] == dois[doi_num]]

NoLigand


In [11]:
# Yields distribution
from rdkit import Chem
import dft_descriptors.featurisation as ft
import dft_descriptors.prepocessing as pp

In [14]:
# Load HTE dataset originally published in :
# Ahneman et al. "Predicting reaction performance in C–N cross-coupling using machine learning." 
# Science 360.6385 (2018): 186-190.
# dataset downloaded from : https://rxn4chemistry.github.io/rxn_yields/data/
df_hte_bh = pd.read_excel("Dreher_and_Doyle_input_data.xlsx")

# Load HTE dataset originally published in :
# Sandfort et al. "A structure-based platform for predicting chemical reactivity." 
# Chem (2020).
# dataset downloaded from : https://rxn4chemistry.github.io/rxn_yields/data/
df_hte_suz = pd.read_excel("aap9112_Data_File_S1.xlsx")

# Load NiCOlit dataset and preprocess
df_dataset = pd.read_csv("../data_csv/Data_test11262021.csv")
df2 = pp.preprocess(df_dataset)
df2 = AL_preprocess(df2)

XLRDError: Excel xlsx file; not supported

In [ ]:
# Featurization of the NiCOlit dataset
X, y, DOIs, mechanisms, origin = ft.process_dataframe_dft(df2, data_path="data_csv/", origin=False)

In [ ]:
# Gather yields for all datasets :
# BH-HTE dataset
y_bh = df_hte_bh["Output"]

# Suzuki-HTE dataset
y_suz = df_hte_suz["Product_Yield_PCT_Area_UV"]

# NiCOlit dataset
y_dataset = y

# Results of the SciFinder query : 
yields = ["<10%", "10-29%", "30-49%", "50-69%", "70-79%", "80-89%", "90-100%"]
yields_lim = [0, 10, 29, 49, 69, 79, 89, 100]
yields_count =  [ 25, 78, 243, 569, 419, 436, 433]

# Simulation of a yield distribution for the SciFinder query : 
Y_sciF = [np.random.randint(yields_lim[i], yields_lim[i+1]) for i in range(len(yields_count)) for j in range(yields_count[i])]

Y = np.concatenate((np.array(y_dataset), np.array(y_suz), np.array(y_bh), np.array(Y_sciF)))

In [ ]:
Origin = np.concatenate((np.array(origin), np.array(["HTE Suzuki" for i in range(len(y_suz))]),
                        np.array(["HTE Buchwald" for i in range(len(y_bh))]),
                       np.array(["SciFinder query" for i in range(len(Y_sciF))])))
Origin2 = np.concatenate((np.array(["NiCO-lit" for i in range(len(y_dataset))]),
                          np.array(["HTE Suzuki" for i in range(len(y_suz))]),
                          np.array(["HTE Buchwald" for i in range(len(y_bh))]),
                         np.array(["SciFinder query" for i in range(len(Y_sciF))])))

In [ ]:
display_df2 =  pd.DataFrame(zip(Y, Origin2), columns =['Yields', 'Origin'])
display_df =  pd.DataFrame(zip(y_dataset, origin, mechanisms), columns =['Yields', 'Origin', 'Mechanism'])

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(10, 10))

#ax[0,1] = plt.subplot(211)

# FIGURE COMP YIELDS DATASETS
sns.violinplot(y="Yields", data=display_df2, x='Origin',  
               kind="swarm", cut=0, ax = ax[0,0], 
               linewidth=1, width=1.2, inner="box") #scale='count')
for tick in ax[0,0].get_xticklabels():
    tick.set_rotation(0)
ax[0,0].set_title("Datasets Yield Distribution")
ax[0,0].set_xlabel("")
ax[0,0].set_ylabel("Yield (\%)")
ax[0,0].set_xticklabels(["NiCOLit", "HTE \n Suzuki", "HTE B-H", "SciFinder\n Query"])


# FIGURE SCOPE/OPT
sns.swarmplot(ax=ax[0,1], y="Yields", data=display_df, x='Origin', color='white', s = 3, 
              linewidth=0.1, dodge=False, edgecolor='black', 
             )
sns.violinplot(y="Yields", data=display_df, x='Origin',  kind="swarm", cut=0, 
               ax = ax[0,1], palette='Blues', scale='count',
              linewidth=1)
ax[0,1].set_title("Scope and Optimisation Yield Distribution\n in the NiCOlit Dataset")
ax[0,1].set_xlabel("")
ax[0,1].set_ylabel("")


# FIGURE SCOPE/OPT in HTE
sns.scatterplot(x='opt', y='sco', hue='yield', data=data_hte, marker="s",
                legend="auto", ax = ax[1,0], palette='flare')

ax[1,0].set_title("B-H HTE Chemical Space Coverage")
ax[1,0].legend(title="Yield (\%)", loc = 'upper right')
ax[1,0].set_xlabel("PC 2 of optimisation subspace")
ax[1,0].set_ylabel("PC 1 of scope subspace")
ax[1,0].set_xticks([])
ax[1,0].set_yticks([])
handles0 = ax[1,0].get_legend_handles_labels()

# FIGURE SCOPE/OPT in NiCOLit
sns.scatterplot(data=data_doi, x='proj scope', y='proj optimisation', 
                hue='yield', ax = ax[1,1], style="data type", legend="auto",
                palette='flare')
ax[1,1].set_title("Academic Publication Chemical Space Coverage")
ax[1,1].set_xlabel("PC 1 of optimisation subspace")
ax[1,1].set_ylabel("PC 1 of scope subspace")
ax[1,1].set_xticks([])
ax[1,1].set_yticks([])

handles = ax[1,1].get_legend_handles_labels()
h0 = handles[0] 
h0.append(handles[0][0])
h0.append(handles[0][0])
h1 = ['Yield (\%)', '15', '30', '45', '60', '75', 'data type', 'optimisation', 'scope', '', '']
ax[1,1].get_legend().remove()
ax[1,1].legend(h0, h1, ncol=2, loc='upper right')

plt.savefig('figure2.svg', dpi=600, format='svg',
     bbox_inches='tight' )